# Events of interest from raw instrumentation logs

In [1]:
import pandas as pd

df = pd.read_csv('lineage_edges_1000.csv')
df.head()

,Job,Action,Dataset,StartTime,EndTime
0,5f1e22a0-440b-4d9a-8b5b-6963ee86da82,read,6f57ebb7-0e27-4503-a4b8-978c251c14b5,4/4/2016 1:55:47 AM,4/4/2016 1:55:50 AM
1,c9f76f52-12e8-4aee-9061-748e13b956e2,read,5719e9da-7f82-470d-bd23-d8ca873f3ea6,4/4/2016 3:11:44 PM,4/4/2016 3:12:44 PM
2,a1c6abfe-a83b-4a31-b112-ed3c9a2b31f4,read,34bae790-b5fc-4f8e-a7d0-1db41f4a11df,4/4/2016 3:18:39 PM,4/4/2016 3:18:59 PM
3,8e0b386e-689b-42e4-acbe-6ee9f0f12dd6,read,0005d127-8a26-4964-a217-84254ac15787,4/4/2016 1:06:07 PM,4/4/2016 1:10:53 PM
4,dba194b3-151a-4fa7-b1a7-773f2523b592,read,14c13e05-0dac-4139-a981-8955c99b5234,4/4/2016 5:53:25 AM,4/4/2016 5:53:35 AM


# Build a Neo4j lineage graph from instrumentation logs

In [ ]:
import lineage_graph_cosmos as lg

graph = lg.build_lineage_graph('lineage_edges_1000.csv')

# Custom visualization of lineage graph using Vis.js

In [ ]:
from scripts.vis import draw

options = {"Dataset": "name", "Job": "uid"}
draw(graph, options)

# Query Neo4j temporal lineage graph using Cypher

## Query results as a Pandas DataFrame

In [ ]:
# Sanity check query.
query = """
    MATCH (n)-[r]->(m)
    RETURN id(n) AS source_id,
           id(r) AS rel_id,
           id(m) AS target_id
"""

results_df = pd.DataFrame(graph.data(query))
print(results_df)

## Copy detection as a reachability query between 2 content similar datasets

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a:\n' % len(paths))
print(paths)

## Copy detection as a reachability query between 2 content similar datasets (monotonically increasing edge timestamps)

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# PageRank

In [ ]:
%load_ext cypher
import networkx as nx
import operator
%matplotlib inline

results = %cypher MATCH p = (a)-[r*]->(b) RETURN p

# Networkx graph.
g = results.get_graph()

#nx.draw(g)
#g.nodes(data=True)

# Print nodes so that we can see their original ids and properties.
print("\nOriginal Nodes:")
print(g.nodes(data=True))

# Node weights for personalized pagerank.
personalize = {}
for node, data in g.nodes(data=True):
    if 'Dataset' in data['labels']:
        personalize[node] = 1
    elif 'Job' in data['labels']:
        # Let's weight each edge by the amount of CPU consumed.
        personalize[node] = data['cpu']
    print("node=%s, data=%s" % (node, data))

print("\nPersonalize Vector:")
print(personalize)

# Transformation from MultiDigraph to Graph for Pagerank calculation.
H = nx.Graph()
for src, dst, edge in g.edges(data=True):
    # Let's weight each edge by the amount of bytes read / written.
    w = edge['data_bytes']
    if H.has_edge(src, dst):
        H[src][dst]['weight'] += w
    else:
        H.add_edge(src, dst, weight=w)

#print("\nPageRank:")
#print(nx.pagerank(H))

print("\nPersonalized PageRank:")
ranks = nx.pagerank(H, personalization=personalize)
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_ranks)

# Data Lineage Temporal Graph (Cosmos)

In [8]:
import time
from py2neo import Graph

# Uncomment this to cleanup all existing graphs.
#query = """
#MATCH (n)
#WITH n limit 100000
#DETACH DELETE n;  
#"""
#graph = Graph()
#graph.data(query)

pwd = os.getcwd()
num_rows = 1000  # Edit this to pick the correct file.
filename = 'file:///%s/lineage_edges_%s.csv' % (pwd, num_rows)

# Load CSV into a neo4j graph.
query = """
USING PERIODIC COMMIT 
LOAD CSV WITH HEADERS FROM '%s'
  AS line
  FIELDTERMINATOR ','
MERGE (src:Job { id: line.Job, name: line.Job, uid: line.Job })
MERGE (dst:Dataset { id: line.Dataset, name: line.Dataset })
FOREACH(t in CASE WHEN line.Action='read' THEN [1] ELSE [] END |
    CREATE (src)-[:READS_FROM {time: line.StartTime}]->(dst))
FOREACH(t in CASE WHEN line.Action='write' THEN [1] ELSE [] END |
    CREATE (src)-[:WRITES_TO {time: line.StartTime}]->(dst))
""" % filename
graph = Graph()
graph.delete_all()

t0 = int(round(time.time() * 1000))
graph.data(query)
t1 = int(round(time.time() * 1000))
print('Time to load %s rows into a neo4j graph: %s ms' % (num_rows, (t1 - t0)))

Time to load 1000 rows into a neo4j graph: 10153 ms


# Visualize graph

In [9]:
from scripts.vis import draw

options = {"Dataset": "name", "Job": "uid"}
draw(graph, options)

# Pagerank and other algos using networkx

In [ ]:
import pandas as pd
import time

num_hops = 3
query = """
    MATCH p=(n)-[r:SENT_EMAIL_TO*%s]->(m)
    RETURN p
""" % num_hops

t0 = int(round(time.time() * 1000))
motifs = graph.data(query)
t1 = int(round(time.time() * 1000))

print('Time to find %s-hop motifs: %s ms' % (num_hops, (t1 - t0)))
#print('%s-hop motifs: %s' % (motifs, num_hops))

%load_ext cypher
import networkx as nx
import operator
%matplotlib inline

# Networkx graph.
t0 = int(round(time.time() * 1000))
results = %cypher MATCH p = (a)-[r]->(b) RETURN p
g = results.get_graph()
#nx.draw(g)
g.nodes(data=True)
t1 = int(round(time.time() * 1000))
print('Time to load it into a networkx graph: %s ms' % (t1 - t0))

t0 = int(round(time.time() * 1000))
#print('In-degree: %s' % g.in_degree())
t1 = int(round(time.time() * 1000))
print('Time to compute in-degree: %s ms' % (t1 - t0))

t0 = int(round(time.time() * 1000))       
centrality = nx.betweenness_centrality(g)
#print('Centrality: %s' % centrality)
t1 = int(round(time.time() * 1000))       
print('Time to compute betweeness centrality: %s ms' % (t1 - t0))


# Print nodes so that we can see their original ids and properties.
#print("\nOriginal Nodes:")
#print(g.nodes(data=True))

# Node weights for personalized pagerank.
t0 = int(round(time.time() * 1000))
personalize = {}
for node, data in g.nodes(data=True):
    if 'Person' in data['labels']:
        personalize[node] = 1
    #print("node=%s, data=%s" % (node, data))
t1 = int(round(time.time() * 1000))
print('Time to create PageRank personalization vector: %s ms' % (t1 - t0))

    
#print("\nPersonalize Vector:")
#print(personalize)

# Transformation from MultiDigraph to Graph for Pagerank calculation.
t0 = int(round(time.time() * 1000))
H = nx.Graph()
for src, dst, edge in g.edges(data=True):
    # Let's weight each edge by the amount of bytes read / written.
    w = int(edge['time'])
    if H.has_edge(src, dst):
        H[src][dst]['weight'] += w
    else:
        H.add_edge(src, dst, weight=w)
t1 = int(round(time.time() * 1000))
print('Time to convert from MultiDigraph to Graph for PageRank calculation: %s ms' % (t1 - t0))


t0 = int(round(time.time() * 1000))       
ranks = nx.pagerank(H, personalization=personalize)
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
t1 = int(round(time.time() * 1000))
print('Time to compute Pagerank: %s ms' % (t1 - t0))

#print("\nPersonalized PageRank:")
#print(sorted_ranks)